In [1]:
######################## think about graph task config types

# source: https://ogb.stanford.edu/docs/graphprop/#ogbg-mol

from ogb.graphproppred import GraphPropPredDataset

dataset = GraphPropPredDataset(name = 'ogbg-molhiv')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]


/home/jimmyzxj/miniconda3/envs/gli_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Downloaded 0.00 GB: 100%|██████████| 3/3 [00:00<00:00,  4.03it/s]


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:00<00:00, 93359.30it/s]


Saving...


In [2]:
import json
import numpy as np
import torch
import scipy.sparse as sparse

In [3]:
print(dataset[0][0].keys())
print(dataset[0][1])

dict_keys(['edge_index', 'edge_feat', 'node_feat', 'num_nodes'])
[0]


In [4]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0
for g, label in tqdm(dataset):
    node_list.append(np.arange(g["num_nodes"]) + num_nodes)  # All the nodes are considered in a single graph
    edges.append(np.stack(g["edge_index"]).T + num_nodes)
    labels.append(label)
    edge_feats.append(g["edge_feat"])
    node_feats.append(g["node_feat"])
    num_nodes += g["num_nodes"]

100%|██████████| 41127/41127 [00:00<00:00, 45962.67it/s]


In [5]:
_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [6]:
_node_list = sparse.lil_matrix((len(dataset), num_nodes))
for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

In [7]:
print(_edges.shape, _labels.shape, _edge_feats.shape, _node_feats.shape, _node_list.shape)

(2259376, 2) (41127,) (2259376, 3) (1049163, 9) (41127, 1049163)


In [10]:
data = {
    "node_feats": _node_feats,
    "graph_class": _labels,
    "edge": _edges,
    "edge_feats": _edge_feats,
    "node_list": _node_list.tocoo(),
}

In [11]:
from gli.utils import save_data
save_data("ogbg-molhiv", **data)

Save all dense arrays to ogbg-molhiv.npz, including ['node_feats', 'graph_class', 'edge', 'edge_feats']
Save sparse matrix node_list to ogbg-molhiv_node_list.sparse.npz


In [12]:
task_data = {
    "train": train_idx,
    "val": valid_idx,
    "test": test_idx
}
save_data("ogbg-molhiv_task", **task_data)

Save all dense arrays to ogbg-molhiv_task.npz, including ['train', 'val', 'test']
